# Practicum: Modern Hopfield Network Spell Checking and Word Recommendation
___
In this practicum problem, we'll implement a modern version of a Hopfield Network, which is a type of recurrent neural network, and we'll use it for spell checking and word recommendation.

* _What are Hopfield Networks_? Hopfield Networks are used for associative memory, where the network can recall a pattern from a partial input. The modern version of Hopfield Networks uses continuous values instead of binary values, and it can store multiple patterns. 
* We'll use the following paper to guide our implementation and analysis: [Ramsauer, H., Schafl, B., Lehner, J., Seidl, P., Widrich, M., Gruber, L., Holzleitner, M., Pavlovi'c, M., Sandve, G.K., Greiff, V., Kreil, D.P., Kopp, M., Klambauer, G., Brandstetter, J., & Hochreiter, S. (2020). Hopfield Networks is All You Need. ArXiv, abs/2008.02217.](https://arxiv.org/abs/2008.02217)

## Tasks
Before we get started, we'll quickly review modern Hopfied Networks. Then, you'll execute the `Run All Cells` command to check if you (or your neighbor) have any code or setup issues. Code issues, then raise your hands - and let's get those fixed!

* __Task 1: Setup, Data, Constants (5 min)__: Let's take 5 minutes to load [a Simpsons character library from Kaggle](https://www.kaggle.com/datasets/kostastokis/simpsons-faces) that our Hopfield network will memorize.
*  __Task 2: Build a Modern Network Model (5 min)__: In this task, we'll formulate the image dataset we give the network and then create a model of a modern Hopfield network. We'll also quickly check to ensure we are doing what we think we are doing.
* __Task 3: Retrieve a memory from the network (30 min)__: In this task, we will retrieve a memory from the modern Hopfield network starting from a random state vector $\mathbf{s}_{\circ}$. We'll corrupt an image (by cutting off some fraction of the image) and then see if the model recovers the correct memory given the corrupted starting point. 

Let's get started!

___

## Background
A modern Hopfield network addresses many of the perceived limitations of the original Hopfield network. The original Hopfield network was limited to binary values and could only store a limited number of patterns. The modern Hopfield network uses continuous values and can store a large number of patterns.
* For a detailed discussion of the key milestones in the development of modern Hopfield networks, check out [Hopfield Networks is All You Need Blog, GitHub.io](https://ml-jku.github.io/hopfield-layers/)

### Algorithm
The user provides a set of memory vectors $\mathbf{X} = \left\{\mathbf{x}_{1}, \mathbf{x}_{2}, \ldots, \mathbf{x}_{m}\right\}$, where $\mathbf{x}_{i} \in \mathbb{R}^{n}$ is a memory vector of size $n$ and $m$ is the number of memory vectors. Further, the user provides an initial _partial memory_ $\mathbf{s}_{\circ} \in \mathbb{R}^{n}$, which is a vector of size $n$ that is a partial version of one of the memory vectors and specifies the _temperature_ $\beta$ of the system.

__Initialize__ the network with the memory vectors $\mathbf{X}$, and the inverse temperature $\beta$. Set current state to the initial state $\mathbf{s} \gets \mathbf{s}_{\circ}$

Until convergence __do__:
   1. Compute the _current_ probability vector defined as $\mathbf{p} = \texttt{softmax}(\beta\cdot\mathbf{X}^{\top}\mathbf{s})$ where $\mathbf{s}$ is the _current_ state vector, and $\mathbf{X}^{\top}$ is the transpose of the memory matrix $\mathbf{X}$.
   2. Compute the _next_ state vector $\mathbf{s}^{\prime} = \mathbf{X}\mathbf{p}$ and the _next_ probability vector $\mathbf{p}^{\prime} = \texttt{softmax}(\beta\cdot\mathbf{X}^{\top}\mathbf{s}^{\prime})$.
   3. If $\mathbf{p}^{\prime}$ is _close_ to $\mathbf{p}$ or we run out of iterations, then __stop__. For example, $\lVert \mathbf{p}^{\prime} - \mathbf{p}\rVert_{2}^{2} \leq \epsilon$ for some small $\epsilon > 0$.
   4. Otherwise, update the state $\mathbf{s} \gets\mathbf{s}^{\prime}$, and __go back to__ step 1.

   
This algorithm is implemented in [the `recover(...)` method](src/Compute.jl).

## Task 1: Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [1]:
include("Include.jl"); # load a bunch of libs, including the ones we need to work with images

### Data
Fill me in